# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Phylsowingdatecorrection

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Phylsowingdatecorrectionmod
    IMPLICIT NONE
CONTAINS

    SUBROUTINE model_phylsowingdatecorrection(sowingDay, &
        latitude, &
        sDsa_sh, &
        rp, &
        sDws, &
        sDsa_nh, &
        p, &
        fixPhyll)
        IMPLICIT NONE
        INTEGER, INTENT(IN) :: sowingDay
        REAL, INTENT(IN) :: latitude
        REAL, INTENT(IN) :: sDsa_sh
        REAL, INTENT(IN) :: rp
        INTEGER, INTENT(IN) :: sDws
        REAL, INTENT(IN) :: sDsa_nh
        REAL, INTENT(IN) :: p
        REAL, INTENT(OUT) :: fixPhyll
        !- Description:
    !            * Title: PhylSowingDateCorrection Model
    !            * Author: Loic Manceau
    !            * Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            * Institution: INRA Montpellier
    !            * Abstract: Correction of the Phyllochron Varietal parameter according to sowing date 
        !- inputs:
    !            * name: sowingDay
    !                          ** description : Day of Year at sowing
    !                          ** parametercategory : species
    !                          ** datatype : INT
    !                          ** min : 1
    !                          ** max : 365
    !                          ** default : 1
    !                          ** unit : d
    !                          ** uri : some url
    !                          ** inputtype : parameter
    !            * name: latitude
    !                          ** description : Latitude
    !                          ** parametercategory : soil
    !                          ** datatype : DOUBLE
    !                          ** min : -90
    !                          ** max : 90
    !                          ** default : 0.0
    !                          ** unit : °
    !                          ** uri : some url
    !                          ** inputtype : parameter
    !            * name: sDsa_sh
    !                          ** description : Sowing date at which Phyllochrone is maximum in southern hemispher
    !                          ** parametercategory : species
    !                          ** inputtype : parameter
    !                          ** datatype : DOUBLE
    !                          ** min : 1
    !                          ** max : 365
    !                          ** default : 1.0
    !                          ** unit : d
    !                          ** uri : some url
    !            * name: rp
    !                          ** description : Rate of change of Phyllochrone with sowing date
    !                          ** parametercategory : species
    !                          ** inputtype : parameter
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 365
    !                          ** default : 0
    !                          ** unit : d-1
    !                          ** uri : some url
    !            * name: sDws
    !                          ** description : Sowing date at which Phyllochrone is minimum
    !                          ** parametercategory : species
    !                          ** datatype : INT
    !                          ** default : 1
    !                          ** min : 1
    !                          ** max : 365
    !                          ** unit : d
    !                          ** uri : some url
    !                          ** inputtype : parameter
    !            * name: sDsa_nh
    !                          ** description : Sowing date at which Phyllochrone is maximum in northern hemispher
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 1.0
    !                          ** min : 1
    !                          ** max : 365
    !                          ** unit : d
    !                          ** uri : some url
    !                          ** inputtype : parameter
    !            * name: p
    !                          ** description : Phyllochron (Varietal parameter)
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** default : 120
    !                          ** min : 0
    !                          ** max : 1000
    !                          ** unit : °C d leaf-1
    !                          ** uri : some url
    !                          ** inputtype : parameter
        !- outputs:
    !            * name: fixPhyll
    !                          ** description :  Phyllochron Varietal parameter 
    !                          ** variablecategory : auxiliary
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 1000
    !                          ** unit : °C d leaf-1
        IF(latitude .LT. 0.0) THEN
            IF(sowingDay .GT. INT(sDsa_sh)) THEN
                fixPhyll = p * (1 - (rp * min((sowingDay - sDsa_sh), float(sDws))))
            ELSE
                fixPhyll = p
            END IF
        ELSE
            IF(sowingDay .LT. INT(sDsa_nh)) THEN
                fixPhyll = p * (1 - (rp * min(sowingDay, sDws)))
            ELSE
                fixPhyll = p
            END IF
        END IF
    END SUBROUTINE model_phylsowingdatecorrection

END MODULE
PROGRAM test
    USE Phylsowingdatecorrectionmod
    INTEGER:: sowingDay
    REAL:: latitude
    REAL:: sDsa_sh
    REAL:: rp
    INTEGER:: sDws
    REAL:: sDsa_nh
    REAL:: p
    REAL:: fixPhyll
    print *, "--------test_test_wheat1_PhylSowingDateCorrection-------"
    sowingDay = 80
    latitude = 33.069
    sDsa_sh = 151
    rp = 0.003
    sDws = 90
    sDsa_nh = 200
    p = 120
    call model_phylsowingdatecorrection(sowingDay, latitude, sDsa_sh, rp,  &
            sDws, sDsa_nh, p, fixPhyll)
    !fixPhyll: 91.2
    print *, "fixPhyll estimated :"
    print *, fixPhyll

END PROGRAM
